# Compare different BBPE tokenizers
The task is to transparently inspect some of the characteristics of different BBPE tokenizers. The tokenizers were trained with different minimum frequency thresholds so they have different vocabularies (and importantly different vocabularies sizes)

The tokenizers are:
- mf3: the standard one we've used so far
- mf30
- mf3000

look for:
- num of trained and special tokens
- average token length in vocabulary
- average chars per token in NIST train set
- vi

In [ ]:
# read the NIST train.msp file
from 
data = 